## Introduction

This Notebook builds the code to scrape the following Wikipedia page, https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M, in order to obtain the neighborhoods in Toronto in a form of DataFrame


In [1]:
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
import requests

## 1. Scrape the Wikipedia page and convert data into a DataFrame

Find the first table in the webpage, scrape all the data and store data in a DataFrame

In [2]:
source  = requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M').text
soup = BeautifulSoup(source, 'lxml')
neigh = soup.find_all('table')[0]
new_table = pd.DataFrame(columns = ['PostalCode', 'Borough', 'Neighborhood'])

temp = pd.DataFrame([('','','')])
temp.columns = ['PostalCode', 'Borough', 'Neighborhood']

for row in neigh.find_all('tr'):
    j = 0
    columns = row.find_all('td')
    for column in columns:
        temp.iloc[0,j] = column.get_text().strip()
        j+=1
    new_table = new_table.append(temp)

#remove the first redandunt row
new_table = new_table[1:]
new_table.reset_index(drop=True, inplace=True)


## 2. Clean the data

Remove row where Boroughs are "Not assigned". If a cell has a borough but a Not assigned neighborhood, then the neighborhood will be the same as the borough. 

In [3]:
# remove rows where Borough not assigned
new_table = new_table[new_table.Borough != 'Not assigned']
new_table.sort_values(by=['PostalCode'], axis=0, inplace=True)

# if Neighborhood is unassigned, set it to Borough
not_assigned = new_table[new_table['Neighborhood']=="Not assigned"]
new_table.iloc[not_assigned.index, 2] = new_table.iloc[not_assigned.index, 1]

new_table

,PostalCode,Borough,Neighborhood
11,M1B,Scarborough,Rouge
12,M1B,Scarborough,Malvern
29,M1C,Scarborough,Port Union
28,M1C,Scarborough,Rouge Hill
27,M1C,Scarborough,Highland Creek
42,M1E,Scarborough,Guildwood
43,M1E,Scarborough,Morningside
44,M1E,Scarborough,West Hill
53,M1G,Scarborough,Scarborough
62,M1H,Scarborough,Cedarbrae


## 3. Build the structured per instruction

Find the unique Postal Codes and then combine the Neighborhood if Borough is the same 

In [4]:
postal = new_table.groupby(['PostalCode', 'Borough']).size().reset_index(name='Neighborhood')

#for matching
codes = new_table['PostalCode'].unique().tolist()

for i in range(len(postal)):
    cut = new_table[new_table['PostalCode']==codes[i]]
    postal.iloc[i,2] = cut.iloc[0,2]
    for j in range(len(cut)-1):
        postal.iloc[i,2] = postal.iloc[i,2] + ', ' + cut.iloc[j+1,2]

print(postal)

    PostalCode           Borough  \
0          M1B       Scarborough   
1          M1C       Scarborough   
2          M1E       Scarborough   
3          M1G       Scarborough   
4          M1H       Scarborough   
5          M1J       Scarborough   
6          M1K       Scarborough   
7          M1L       Scarborough   
8          M1M       Scarborough   
9          M1N       Scarborough   
10         M1P       Scarborough   
11         M1R       Scarborough   
12         M1S       Scarborough   
13         M1T       Scarborough   
14         M1V       Scarborough   
15         M1W       Scarborough   
16         M1X       Scarborough   
17         M2H        North York   
18         M2J        North York   
19         M2K        North York   
20         M2L        North York   
21         M2M        North York   
22         M2N        North York   
23         M2P        North York   
24         M2R        North York   
25         M3A        North York   
26         M3B        North 

In [5]:
print(postal.shape)

(103, 3)
